Evaluate the models

# Stochastic Adversarial Training (StochAT)

### SoTA

vanila SGD: 
MNIST - 99%+ (most cnns), CIFAR10 - 93%+ (resnet18), 96%+ (wideresnet) 

MNIST:

adversarial attacks: 
l-inf @ eps = 80/255 @20 steps: TRADES - 96.07% - (4 layer cnn), MART 96.4%, MMA 95.5%, PGD - 96.01% - (4 layer cnn)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MMA, PGD

CIFAR10:

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
TRADES 53-56% - (WRN-34-10), MART 57-58% (WRN-34-10), MMA 47%, PGD 48% - (WRN-32-10)// 49% - (WRN-34-10), Std - 0.03%
https://openreview.net/pdf?id=rklOg6EFwS (Table 4)

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
[ResNet10] TRADES 45.4%, MART 46.6%, MMA 37.26%, PGD 42.27%, Std 0.14%

Benign accuracies: TRADES 84.92%, MART 83.62%, MMA 84.36, PGD 87.14%, Std 95.8% [wideresnet]
https://openreview.net/pdf?id=Ms9zjhVB5R (Table 1)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MART, MMA, PGD

TBD: CWinf attacks

## Pretrained models for comparison

download pretrained models and place in ../trainedmodels/MNIST or ../trainedmodels/CIFAR10 respectively

### TRADES :
https://github.com/yaodongyu/TRADES (MNIST: small cnn, CIFAR10: WideResNet34)
### MMA : 
https://github.com/BorealisAI/mma_training (MNIST: lenet5, CIFAR10: WideResNet28)
### MART :
 https://github.com/YisenWang/MART (CIFAR10: ResNet18 and WideResNet34)

## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from multiprocessing import cpu_count
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import olympic
from typing import Union, Callable, Tuple
import sys
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
from functional import boundary, iterated_fgsm, local_search, pgd, entropySmoothing
from ESGD_utils import *
import pickle
import time
import torch.backends.cudnn as cudnn
import argparse, math, random
import ESGD_optim
from trades import trades_loss

In [2]:
from torch.autograd import Variable

In [3]:
from torch.optim.lr_scheduler import StepLR

In [4]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [5]:
DEVICE

'cuda'

# LOAD DATA

In [6]:
#place data folders outside working directory

In [7]:
kwargs = {'num_workers': 1, 'pin_memory': True}

In [8]:
dataset = 'CIFAR10' # [MNIST, CIFAR10]
transform = transforms.Compose([
    transforms.ToTensor(),
])
bsz = 128
if dataset == 'MNIST':
    train = datasets.MNIST('../../data/MNIST', train=True, transform=transform, download=True)
    val = datasets.MNIST('../../data/MNIST', train=False, transform=transform, download=True)
elif dataset == 'CIFAR10':
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])

    train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform_train, download=True)
    val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform_test, download=True)
     
train_loader = DataLoader(train, batch_size=128, shuffle=True, **kwargs)
val_loader = DataLoader(val, batch_size=128, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


# INITIALIZE NETWORK

In [9]:
if dataset=='MNIST':
    from small_cnn import SmallCNN   
    Net = SmallCNN
    NetName = 'SmallCNN'

In [10]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    Net = WideResNet
    NetName = 'WideResNet'

# RANDOM SEED 

In [11]:
args = {}
args['test_batch_size'] = 128
args['no_cuda'] = False
args['epsilon'] = 0.031
args['num_steps'] = 10
args['step_size'] = 0.007
args['random'] =True,
args['white_box_attack']=True
args['log_interval'] = 100
args['beta'] = 6.0
args['seed'] = 1
args['lr'] = 0.1
args['momentum'] = 0.9
args['epochs'] = 76
args['batch_size'] = 128
args['save_freq'] = 3
args['weight_decay'] = 2e-4

In [12]:
seed = args['seed']
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [13]:
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy

In [14]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [15]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=0.031,
                  num_steps=20,
                  step_size=0.003):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args['random']:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(DEVICE)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd (white-box): ', err_pgd)
    return err, err_pgd

def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_acc_total: ', 100-natural_err_total.item()/100)
    print('robust_acc_total: ', 100-robust_err_total.item()/100)

In [16]:
if not os.path.exists('../trainedmodels'):
    os.makedirs('../trainedmodels')

In [17]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [18]:
def train_adversarial(method,model, device, train_loader, optimizer, epoch,adversary,k,step,eps,norm,random):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ypred = model(data)
        
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss
        loss = method(model,optimizer,sgd_loss,data,target,epoch,adversary,k,step,eps,norm,random)

# TRAIN MODEL USING TRADES

In [26]:
ls ../WRN_TRADES

model-nn-epoch10.pt  model-nn-epoch61.pt
model-nn-epoch15.pt  model-nn-epoch64.pt
model-nn-epoch20.pt  opt-nn-checkpoint_epoch26.tar
model-nn-epoch25.pt  opt-nn-checkpoint_epoch30.tar
model-nn-epoch26.pt  opt-nn-checkpoint_epoch35.tar
model-nn-epoch30.pt  opt-nn-checkpoint_epoch40.tar
model-nn-epoch35.pt  opt-nn-checkpoint_epoch45.tar
model-nn-epoch40.pt  opt-nn-checkpoint_epoch50.tar
model-nn-epoch45.pt  opt-nn-checkpoint_epoch52.tar
model-nn-epoch50.pt  opt-nn-checkpoint_epoch55.tar
model-nn-epoch52.pt  opt-nn-checkpoint_epoch58.tar
model-nn-epoch55.pt  opt-nn-checkpoint_epoch61.tar
model-nn-epoch58.pt  opt-nn-checkpoint_epoch64.tar
model-nn-epoch5.pt


In [27]:
model_TRADESInf = Net().to(DEVICE)
model_TRADESInf.load_state_dict(torch.load('../WRN_TRADES/model-nn-epoch64.pt'))

<All keys matched successfully>

In [21]:
#eval_train(model_TRADESInf, DEVICE, train_loader)
#eval_adv_test_whitebox(model_TRADESInf, DEVICE, val_loader)

In [28]:
def train_trades(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=args['step_size'],
                           epsilon=args['epsilon'],
                           perturb_steps=args['num_steps'],
                           beta=args['beta'])

        loss.backward()
        optimizer.step()


        # print progress
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [29]:
def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate"""
    lr = args['lr']
    if epoch >= 55:
        lr = args['lr'] * 0.1
    if epoch >= 64:
        lr = args['lr'] * 0.01
    if epoch >= 100:
        lr = args['lr'] * 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [30]:
model_dir = '../WRN_TRADES'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [31]:
#### args['attack'] = 'l_inf' 
## initialize model
#model_TRADESInf = Net().to(DEVICE)
## training params
optimizer = optim.SGD(model_TRADESInf.parameters(), lr=args['lr'], momentum=args['momentum'], weight_decay=args['weight_decay'])
epochs = 76
## train model

for epoch in range(65, epochs + 1):
    print('Epoch:',epoch)
    # adjust learning rate for SGD
    adjust_learning_rate(optimizer, epoch)
    for param_group in optimizer.param_groups:
        print('lr:',param_group['lr'])
    # adversarial training
    train_trades(args, model_TRADESInf, DEVICE, train_loader, optimizer, epoch)

    # evaluation on natural examples
    if (epoch-1) % 1 == 0:
        print('================================================================')
        eval_train(model_TRADESInf, DEVICE, train_loader)
        eval_adv_test_whitebox(model_TRADESInf, DEVICE, val_loader)
        print('================================================================')

    if (epoch-1) % args['save_freq'] == 0:
        torch.save(model_TRADESInf.state_dict(),
               os.path.join(model_dir, 'model-nn-epoch{}.pt'.format(epoch)))
        torch.save(optimizer.state_dict(),
                       os.path.join(model_dir, 'opt-nn-checkpoint_epoch{}.tar'.format(epoch)))
## save model
modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADESInf_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
torch.save(model_TRADESInf,modelname)

Epoch: 65
lr: 0.001


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 65 [0/50000 (0%)]	Loss: 0.718938
Train Epoch: 65 [12800/50000 (26%)]	Loss: 0.720713
Train Epoch: 65 [25600/50000 (51%)]	Loss: 0.718890
Train Epoch: 65 [38400/50000 (77%)]	Loss: 0.868327
Training: Average loss: 0.4007, Accuracy: 48081/50000 (96%)
natural_acc_total:  83.71000000000001
robust_acc_total:  53.79
Epoch: 66
lr: 0.001
Train Epoch: 66 [0/50000 (0%)]	Loss: 0.885656
Train Epoch: 66 [12800/50000 (26%)]	Loss: 0.886515
Train Epoch: 66 [25600/50000 (51%)]	Loss: 0.738653
Train Epoch: 66 [38400/50000 (77%)]	Loss: 0.758270
Training: Average loss: 0.4088, Accuracy: 47896/50000 (96%)
natural_acc_total:  83.75
robust_acc_total:  53.95
Epoch: 67
lr: 0.001
Train Epoch: 67 [0/50000 (0%)]	Loss: 0.728145
Train Epoch: 67 [12800/50000 (26%)]	Loss: 0.722494
Train Epoch: 67 [25600/50000 (51%)]	Loss: 0.807068
Train Epoch: 67 [38400/50000 (77%)]	Loss: 0.786618
Training: Average loss: 0.3834, Accuracy: 48135/50000 (96%)
natural_acc_total:  84.09
robust_acc_total:  53.36
Epoch: 68
lr: 0.00

NameError: name 'lr' is not defined

In [ ]:
eval_train(model_TRADESInf, DEVICE, train_loader)
eval_test(model_TRADESInf, DEVICE, val_loader)

In [ ]:
model_TRADESInf = Net().to(DEVICE)
model_TRADESInf.load_state_dict(torch.load('../WRN_TRADES/model-nn-epoch5.pt'))